#**RAG LIC Q/A BOT:**


We aim to develop an AI-powered RAG chatbot for LIC policyholders, automating client inquiries and providing comprehensive policy details through natural language interactions.






**1) Installing necessary libraries**




In [15]:
!pip install langchain
!pip install PyPDF2
!pip install python-dotenv
!pip install streamlit
!pip install openai
!pip install faiss-cpu
!pip install langchain_community
!pip install sentence-transformers
!pip install langchainhub
!pip install langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.3.2
    Uninstalling google-generativeai-0.3.2:
      Successfully uninstalled google-generativeai-0.3.2


**2) Importing necessary libraries**

In [16]:
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
import PyPDF2
import re
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
from langchain import hub
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
import pandas as pd
from langchain.evaluation import load_evaluator
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.evaluation import EvaluatorType
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough



**3) The document is in the form of a PDF. Use PyPDF2 to extract the content in the form of text.**

In [17]:


#regex_pattern = r'\b[Gr8AmbitionZ.comShortnotes on LIC Plans  - powered by Gr8Ambition Z.com]+[Shortnotes on  LIC  Insurance Plans  Powered by www.Gr8 Ambition Z.com your A to Z competitive exam guide]+[LIC Insurance yourcompetitive exam guide ]\b'

extracted_text =''

# Open the PDF file in read-binary mode ('rb')
with open('/content/LIC Plans .pdf', 'rb') as file:
    # Create a PdfFileReader object
    pdf_reader = PyPDF2.PdfReader(file)

    # Display the number of pages in the PDF file
    print(f"Number of pages: {len(pdf_reader.pages)}")

    # Assuming you've already opened the PDF file
    for i in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[i]
        extracted_text += page.extract_text()


print(extracted_text)




Number of pages: 38
Shortnotes on  
LIC  
Insurance Plans  
 
Powered by 
www.Gr8 Ambition Z.com 
your A to Z competitive exam guide  
Gr8AmbitionZ.comShortnotes on LIC Plans  - powered by Gr8Ambition Z.com 
 1  
he first name that comes to one’s mind when one thinks of life insurance india is that of LIC. Life Insurance 
Corporation of India (LIC)  is the only public sector life insurance company in India. It was formed on 1st 
September 1956 with the merger of more  than 200 insurance companies and provident societies. LIC has its 
headquarters in Mumbai, the commercial capital of India and currently functions with 8 zonal offices  and 113 divisional 
offices. It is the largest life insurance company in India with over 2000 branches and over 14 lakh agents to solicit life 
insurance business in the country. It has a phenomenal presence in both urban and rural India. The m otto of the company 
is "Yogakshemam Vahamyaham " which means - Your welfare is our responsibility.  
LIC command

**4) Have the list of all policies in the PDF. Chunking data based on fixed sizes results in mixed data within chunks, confusing the large language model during answer generation. To avoid this, make the chunks manually by tracing the index numbers of all policies, sub-policies and storing them in a dictionary**

In [19]:
def find_the_index(text, target_line):
#     lines = text.split('\n')
#     try:
    index = text.find(target_line)
#     return lines[index+1:index+8]  # Fetch 7 lines after the target line
    return index
#     except ValueError:
#         return "Target not found"


db_dict = {}
target_line = [
    "he first name",
    "Ltd. Payment with Profits",
    "Jeevan Mitra Double",
    "Jeevan Saathi :",
    "Marriage  Endowment with Profits",
    "New Janaraksha  :",
    "Jeevan Chhaya :",
    "Jeevan Mitra Triple Cover :",
    "Jeevan Anand :",
    "Jeevan Shree",
    "Jeevan Pramukh",
    "Jeevan Anurag :",
    "Jeevan Amrit",
    "Money Back Plans are ideal",
    "Money Back (25 Years)",
    "Jeevan Surabhi (15 Years)",
    "Jeevan Surabhi  (20 years)",
    "Jeevan Surabhi  (25 years)",
    "Bima Bachat (9, 12 & 15 yrs)",
    "Jeevan Tarang",
    "New Bima Gold (12,16 & 20 yrs)",
    "Jeevan Bharati -1 (15 & 20 yrs)",
    "Child Insurance Plans help",
    "This plan is specially designed for a child's marriage or education. Premium has",
    "Komal Jeevan  :",
    "Jeevan Anurag :",
    "Child Career Plan",
    "Child Future Plan",
    "This  is a conventional with profits plan",
    "Pension Plans foresee financial",
    "Term Insurance is a basic life",
    "Anmol Jeevan",
    "Market Linked Plans are also known as Unit Linked Plans or ULIPS",
    "Special Plans are Limited edition plans",
    "Jeevan Saral",
    "Health Plans :",
    "Policy for Handicapped",
    "o Jeevan Vishwas",
    "All the Best for your Exam"
]

for i in target_line:
    result = find_the_index(extracted_text, i)
    db_dict[i]= result
print(db_dict)


{'he first name': 193, 'Ltd. Payment with Profits': 4524, 'Jeevan Mitra Double': 5975, 'Jeevan Saathi :': 7331, 'Marriage  Endowment with Profits': 9677, 'New Janaraksha  :': 11650, 'Jeevan Chhaya :': 13177, 'Jeevan Mitra Triple Cover :': 15165, 'Jeevan Anand :': 16522, 'Jeevan Shree': 18401, 'Jeevan Pramukh': 19328, 'Jeevan Anurag :': 20391, 'Jeevan Amrit': 22957, 'Money Back Plans are ideal': 24964, 'Money Back (25 Years)': 26893, 'Jeevan Surabhi (15 Years)': 28357, 'Jeevan Surabhi  (20 years)': 30728, 'Jeevan Surabhi  (25 years)': 33149, 'Bima Bachat (9, 12 & 15 yrs)': 35527, 'Jeevan Tarang': 37331, 'New Bima Gold (12,16 & 20 yrs)': 39078, 'Jeevan Bharati -1 (15 & 20 yrs)': 41548, 'Child Insurance Plans help': 43373, "This plan is specially designed for a child's marriage or education. Premium has": 46632, 'Komal Jeevan  :': 48599, 'Child Career Plan': 53901, 'Child Future Plan': 57698, 'This  is a conventional with profits plan': 60659, 'Pension Plans foresee financial': 63436, 'Te

**5) The index numbers of all the policies are stored in db_list in a sequence. Their corresponding text was stored in my_list1 in the same order.**

In [20]:
db_list = []
my_list1=[]


for key,value in db_dict.items():
    db_list.append(value)

for i in range(len(db_list)-1):
    text = extracted_text[db_list[i]:db_list[i+1]]
    my_list1.append(text)

print(len(my_list1))



37


**6) Use FAISS (Facebook AI Similarity Search) as vector database to store the vector embeddings. Use Huggingface embeddings to convert the text into vector embeddings and storing these embeddings in FAISS database.**

In [21]:
embeddings = HuggingFaceEmbeddings()
# embeddings = OpenAIEmbeddings()

fas = FAISS.from_texts(my_list1, embeddings)


**7) Load HuggingFace endpoints for mistralai/Mistral-7B-Instruct-v0.1. A custom prompt template must be created to direct the structure of context, question and answer. FAISS retriever iss used to retreiver similar chunks by calculating L2 distance similarity metric. RAG chain is build incorporating the context, prompt and llm.**

In [22]:

HUGGINGFACEHUB_API_TOKEN ='YOUR_HUGGINGFACE_API_KEY'

prompt = hub.pull("rlm/rag-prompt")


custom_prompt_template = """
Context: {context}
Question: {question}
Answer:
"""


repo_id = "mistralai/Mistral-7B-Instruct-v0.1"

llm = HuggingFaceEndpoint(repo_id=repo_id,huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,temperature=0.1)
custom_prompt = PromptTemplate.from_template(custom_prompt_template)



def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = fas.as_retriever(search_type="similarity", search_kwargs={"k":2})




rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_prompt
    | llm
    | StrOutputParser()
)



Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


**8) Example implementation**

In [23]:
rag_chain.invoke("What is the minimum term for Child Career Plan?")


'Answer: The minimum term for Child Career Plan is 11 years.'

**9) Loading the benchmarking dataset containing questions and ground truth in db variable.**

In [24]:


excel_file  = pd.ExcelFile('/content/rag_questions.xlsx')
sheet_name = 'Sheet1'  # Change this to the name of your sheet or its index
df = pd.read_excel(excel_file, sheet_name=sheet_name)
df.head(5)



,Unnamed: 0.1,Unnamed: 0,Questions,Ground_truth,Extracted chunk,Generated answer
0,0,0,"What is the significance of LIC's motto ""Yogak...","The motto ""Yogakshemam Vahamyaham"" translates ...",NaN,NaN
1,1,1,How does LIC's market share compare to that of...,LIC commands a market share of 72% of all new ...,NaN,NaN
2,2,2,What are some key features of the Jeevan Anand...,The Jeevan Anand plan combines aspects of both...,NaN,NaN
3,3,3,How does the New Janaraksha plan cater to indi...,The New Janaraksha plan is tailored for indivi...,NaN,NaN
4,4,4,What is the Maturity Benefit provided by the J...,The Maturity Benefit under the Jeevan Mitra Tr...,NaN,NaN


##**Retrieving**

**10) For every question , top 2 similar chunks are extracted from the retreiver and added into the dataframe. ***

In [25]:


questions_list = df['Questions'].to_list()



str1=''
ret_doc = []
row = 1
column = 'Extracted chunk'
for question in questions_list:
  docs = fas.similarity_search_with_relevance_scores(question)
  ret_doc.append(docs[0][0].page_content + " " + docs[1][0].page_content)

row = 0
column = 'Extracted chunk'
for i in ret_doc:
  df.at[row, column] = i
  row = row + 1


/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='he first name that comes to one’s mind when one thinks of life insurance india is that of LIC. Life Insurance \nCorporation of India (LIC)  is the only public sector life insurance company in India. It was formed on 1st \nSeptember 1956 with the merger of more  than 200 insurance companies and provident societies. LIC has its \nheadquarters in Mumbai, the commercial capital of India and currently functions with 8 zonal offices  and 113 divisional \noffices. It is the largest life insurance company in India with over 2000 branches and over 14 lakh agents to solicit life \ninsurance business in the country. It has a phenomenal presence in both urban and rural India. The m otto of the company \nis "Yogakshemam Vahamyaham " which means - Your welfare is our responsibility.  \nLIC commands market share of 72 per cent of all new polic

In [26]:
df.head(15)

,Unnamed: 0.1,Unnamed: 0,Questions,Ground_truth,Extracted chunk,Generated answer
0,0,0,"What is the significance of LIC's motto ""Yogak...","The motto ""Yogakshemam Vahamyaham"" translates ...",he first name that comes to one’s mind when on...,NaN
1,1,1,How does LIC's market share compare to that of...,LIC commands a market share of 72% of all new ...,he first name that comes to one’s mind when on...,NaN
2,2,2,What are some key features of the Jeevan Anand...,The Jeevan Anand plan combines aspects of both...,Jeevan Anand : \nThis plan is the h ighest se...,NaN
3,3,3,How does the New Janaraksha plan cater to indi...,The New Janaraksha plan is tailored for indivi...,New Janaraksha : \nThis plan is best suited f...,NaN
4,4,4,What is the Maturity Benefit provided by the J...,The Maturity Benefit under the Jeevan Mitra Tr...,Jeevan Mitra Triple Cover : \nThis plan is b...,NaN
5,5,5,How does the Death Benefit differ for accident...,In the event of the policyholder's demise in a...,Jeevan Mitra Triple Cover : \nThis plan is b...,NaN
6,6,6,What are the key features of the Maturity Bene...,The Maturity Benefit includes two components. ...,Jeevan Anurag : \nThis plan is specially desi...,NaN
7,7,7,Describe the Death Benefit under the Jeevan An...,Upon the death of the life assured during the ...,Jeevan Anurag : \nThis plan is specially desi...,NaN
8,8,8,What distinguishes the Child Career Plan from ...,The Child Career Plan is uniquely designed to ...,Child Future Plan \nThis plan is specifically...,NaN
9,9,9,What are the eligibility criteria and key bene...,The Komal Jeevan plan is an ideal gift for a c...,Anmol Jeevan \nThis plan is a pure term insu...,NaN


##**Generating**

**11) Answers for each question are generated by LLM and stored in the dataframe.**

In [27]:
questions = df['Questions'].to_list()
answers = []
for question in questions:
  answer = rag_chain.invoke(str(question))
  answers.append(answer)





In [28]:
row = 0
column = 'Generated answer'
for answer in answers:
  df.at[row, column] = answer
  row = row +1

In [29]:
df.head(20)

,Unnamed: 0.1,Unnamed: 0,Questions,Ground_truth,Extracted chunk,Generated answer
0,0,0,"What is the significance of LIC's motto ""Yogak...","The motto ""Yogakshemam Vahamyaham"" translates ...",he first name that comes to one’s mind when on...,"The motto of LIC, ""Yogakshemam Vahamyaham"", me..."
1,1,1,How does LIC's market share compare to that of...,LIC commands a market share of 72% of all new ...,he first name that comes to one’s mind when on...,\nLIC commands a market share of 72 per cent o...
2,2,2,What are some key features of the Jeevan Anand...,The Jeevan Anand plan combines aspects of both...,Jeevan Anand : \nThis plan is the h ighest se...,The Jeevan Anand plan is a combination of Whol...
3,3,3,How does the New Janaraksha plan cater to indi...,The New Janaraksha plan is tailored for indivi...,New Janaraksha : \nThis plan is best suited f...,\nThe New Janaraksha plan is designed to cater...
4,4,4,What is the Maturity Benefit provided by the J...,The Maturity Benefit under the Jeevan Mitra Tr...,Jeevan Mitra Triple Cover : \nThis plan is b...,The Maturity Benefit provided by the Jeevan Mi...
5,5,5,How does the Death Benefit differ for accident...,In the event of the policyholder's demise in a...,Jeevan Mitra Triple Cover : \nThis plan is b...,"Under the Jeevan Mitra Triple Cover plan, the ..."
6,6,6,What are the key features of the Maturity Bene...,The Maturity Benefit includes two components. ...,Jeevan Anurag : \nThis plan is specially desi...,"Under the Jeevan Anurag plan, the Maturity Ben..."
7,7,7,Describe the Death Benefit under the Jeevan An...,Upon the death of the life assured during the ...,Jeevan Anurag : \nThis plan is specially desi...,The Death Benefit under the Jeevan Anurag plan...
8,8,8,What distinguishes the Child Career Plan from ...,The Child Career Plan is uniquely designed to ...,Child Future Plan \nThis plan is specifically...,Answer: The Child Career Plan is specifically ...
9,9,9,What are the eligibility criteria and key bene...,The Komal Jeevan plan is an ideal gift for a c...,Anmol Jeevan \nThis plan is a pure term insu...,The Komal Jeevan plan is a Money Back plan for...


In [ ]:
# df.to_excel('/content/Final_rag_theory.xlsx')

#**Benchmarking**

**12) Use Langchain Evaluator to evaluate the chunks and the generated answer. Langchain uses GPT-4 as the default LLM for benchmarking. It can be customized by using Gemini-Pro as the LLM and did the evaluation of the RAG system. We have used 3 metrices, nameky correctness, conciseness and context relevance**

In [30]:

# Set the GOOGLE_API_KEY environment variable
os.environ['GOOGLE_API_KEY']='YOUR_GOOGLE_API'

# Create the Gemini instance
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                            temperature=0.7,convert_system_message_to_human=True
                                )

# Load the evaluator with the Gemini instance

conciseness_evaluator = load_evaluator("criteria", llm=llm, criteria="conciseness")

correctness_evaluator = load_evaluator("labeled_criteria", llm=llm, criteria="correctness")





truth = df['Ground_truth'].to_list()
chunks = df['Extracted chunk'].to_list()
questions = df["Questions"].to_list()
generated_answer = df['Generated answer'].to_list()

correctness_checker = []
conciseness_checker = []
for i in range(20):
  conciseness_result = conciseness_evaluator.evaluate_strings(
      prediction=generated_answer[i],
      input=questions[i],
  )
  conciseness_checker.append(conciseness_result)

  correctness_result = correctness_evaluator.evaluate_strings(
      input = questions[i],
      prediction = generated_answer[i],
      reference = truth[i],
  )

  correctness_checker.append(correctness_result)








In [31]:
correctness_final = []
conciseness_final=[]
for i in range(20):
  last_list1 = list(correctness_checker[i].values())[-1]
  last_list2 = list(conciseness_checker[i].values())[-1]
  correctness_final.append(last_list1)
  conciseness_final.append(last_list2)

print((correctness_final))
print((conciseness_final))

[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, None, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]


**13) Conciseness and Correctness are default metrices available in Langchain evaluator.**

**Conciseness** refers to the quality of being clear and brief in communication. In the context of the criteria evaluator, it assesses whether the model response is succinct and to the point, providing a clear and efficient answer without unnecessary verbosity or redundancy, indicated by a binary compliance score.

**Correctness** pertains to the accuracy and precision of information conveyed in the model's response. In the evaluation process, correctness criteria necessitate reference labels, enabling comparison between the model output and the established standards, ensuring the response aligns closely with expected or accurate information.



These values for each question are added to the dataframe.

In [32]:
data = pd.DataFrame({'Conciseness': conciseness_final, 'Correctness': correctness_final})
data = data.reset_index(drop=True)
df = df.reset_index(drop=True)
df = pd.concat([df, data],axis=1,join='outer')
# df = df.append(data, ignore_index=True)






In [33]:
df.head(20)

,Unnamed: 0.1,Unnamed: 0,Questions,Ground_truth,Extracted chunk,Generated answer,Conciseness,Correctness
0,0,0,"What is the significance of LIC's motto ""Yogak...","The motto ""Yogakshemam Vahamyaham"" translates ...",he first name that comes to one’s mind when on...,"The motto of LIC, ""Yogakshemam Vahamyaham"", me...",1,1.0
1,1,1,How does LIC's market share compare to that of...,LIC commands a market share of 72% of all new ...,he first name that comes to one’s mind when on...,\nLIC commands a market share of 72 per cent o...,1,1.0
2,2,2,What are some key features of the Jeevan Anand...,The Jeevan Anand plan combines aspects of both...,Jeevan Anand : \nThis plan is the h ighest se...,The Jeevan Anand plan is a combination of Whol...,1,1.0
3,3,3,How does the New Janaraksha plan cater to indi...,The New Janaraksha plan is tailored for indivi...,New Janaraksha : \nThis plan is best suited f...,\nThe New Janaraksha plan is designed to cater...,1,1.0
4,4,4,What is the Maturity Benefit provided by the J...,The Maturity Benefit under the Jeevan Mitra Tr...,Jeevan Mitra Triple Cover : \nThis plan is b...,The Maturity Benefit provided by the Jeevan Mi...,1,1.0
5,5,5,How does the Death Benefit differ for accident...,In the event of the policyholder's demise in a...,Jeevan Mitra Triple Cover : \nThis plan is b...,"Under the Jeevan Mitra Triple Cover plan, the ...",1,1.0
6,6,6,What are the key features of the Maturity Bene...,The Maturity Benefit includes two components. ...,Jeevan Anurag : \nThis plan is specially desi...,"Under the Jeevan Anurag plan, the Maturity Ben...",0,0.0
7,7,7,Describe the Death Benefit under the Jeevan An...,Upon the death of the life assured during the ...,Jeevan Anurag : \nThis plan is specially desi...,The Death Benefit under the Jeevan Anurag plan...,1,1.0
8,8,8,What distinguishes the Child Career Plan from ...,The Child Career Plan is uniquely designed to ...,Child Future Plan \nThis plan is specifically...,Answer: The Child Career Plan is specifically ...,0,1.0
9,9,9,What are the eligibility criteria and key bene...,The Komal Jeevan plan is an ideal gift for a c...,Anmol Jeevan \nThis plan is a pure term insu...,The Komal Jeevan plan is a Money Back plan for...,1,1.0


In [34]:
conciseness_value = df.Conciseness.mean()
correctness_value = df.Correctness.mean()

print(conciseness_value)
print(correctness_value)

0.8
0.7894736842105263


**14) Context Relevance metric is a custom made metric which is not present in Langchain Evaluator. To find the context relevancy, we have passed the retrieved chunk and question. It will calculate the revelance  of the retrieved chunk as compared to the question asked by the user.**

**Context Relevance** is a metric that evaluates how well the retrieved context matches the question, with a score ranging from 0 to 1. It assesses the relevance of the context by identifying sentences within it that are crucial for answering the question. The evaluator will try to find relevant sentences in the retrieved chunk as compared to the given question. Higher scores indicate better relevancy, aiming for contexts that exclusively contain essential information to address the query.

In [35]:
context_relevance =[]


for i in range(20):


  custom_criteria = {
      "criteria" : "Are you able to extract a relevant string from prediction related to input ?"
  }

  input = questions[i]

  prediction = chunks[i]
  eval_chain = load_evaluator(
      EvaluatorType.CRITERIA,
      criteria=custom_criteria,
      llm=llm
  )

  eval_result = eval_chain.evaluate_strings(input = input, prediction=prediction)

  context_relevance.append(eval_result)



In [36]:
context_relevance_final=[]
for i in range(20):
  last_list1 = list(context_relevance[i].values())[-1]
  context_relevance_final.append(last_list1)


print(context_relevance_final)

[1, 1, 0, 1, 1, None, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0]


In [37]:
data = pd.DataFrame({'context_relevance': context_relevance_final})
data = data.reset_index(drop=True)
df = pd.concat([df, data],axis=1,join='outer')

In [38]:
df['context_relevance'].mean()

0.631578947368421

In [39]:
df.head(15)

,Unnamed: 0.1,Unnamed: 0,Questions,Ground_truth,Extracted chunk,Generated answer,Conciseness,Correctness,context_relevance
0,0,0,"What is the significance of LIC's motto ""Yogak...","The motto ""Yogakshemam Vahamyaham"" translates ...",he first name that comes to one’s mind when on...,"The motto of LIC, ""Yogakshemam Vahamyaham"", me...",1,1.0,1.0
1,1,1,How does LIC's market share compare to that of...,LIC commands a market share of 72% of all new ...,he first name that comes to one’s mind when on...,\nLIC commands a market share of 72 per cent o...,1,1.0,1.0
2,2,2,What are some key features of the Jeevan Anand...,The Jeevan Anand plan combines aspects of both...,Jeevan Anand : \nThis plan is the h ighest se...,The Jeevan Anand plan is a combination of Whol...,1,1.0,0.0
3,3,3,How does the New Janaraksha plan cater to indi...,The New Janaraksha plan is tailored for indivi...,New Janaraksha : \nThis plan is best suited f...,\nThe New Janaraksha plan is designed to cater...,1,1.0,1.0
4,4,4,What is the Maturity Benefit provided by the J...,The Maturity Benefit under the Jeevan Mitra Tr...,Jeevan Mitra Triple Cover : \nThis plan is b...,The Maturity Benefit provided by the Jeevan Mi...,1,1.0,1.0
5,5,5,How does the Death Benefit differ for accident...,In the event of the policyholder's demise in a...,Jeevan Mitra Triple Cover : \nThis plan is b...,"Under the Jeevan Mitra Triple Cover plan, the ...",1,1.0,NaN
6,6,6,What are the key features of the Maturity Bene...,The Maturity Benefit includes two components. ...,Jeevan Anurag : \nThis plan is specially desi...,"Under the Jeevan Anurag plan, the Maturity Ben...",0,0.0,1.0
7,7,7,Describe the Death Benefit under the Jeevan An...,Upon the death of the life assured during the ...,Jeevan Anurag : \nThis plan is specially desi...,The Death Benefit under the Jeevan Anurag plan...,1,1.0,1.0
8,8,8,What distinguishes the Child Career Plan from ...,The Child Career Plan is uniquely designed to ...,Child Future Plan \nThis plan is specifically...,Answer: The Child Career Plan is specifically ...,0,1.0,0.0
9,9,9,What are the eligibility criteria and key bene...,The Komal Jeevan plan is an ideal gift for a c...,Anmol Jeevan \nThis plan is a pure term insu...,The Komal Jeevan plan is a Money Back plan for...,1,1.0,0.0


**15) Dataframe is loaded into excel file**

In [40]:
df.to_excel('/content/Final_rag_theory.xlsx')